In [91]:
from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
''')

# Imports

In [166]:
import pandas as pd
import geopandas as gpd
import sqlalchemy
import seaborn as sns
from shapely import wkt
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
from sqlalchemy import text
import numpy as np
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json
import plotly.express as px
import folium
from folium.plugins import HeatMap

# SQL Connection Functions

In [358]:
credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['database']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

crs = 'EPSG:4326'
srid = 4326

In [359]:
db, conn = pgconnect(credentials)

Connected successfully.


In [360]:
conn.execute(text("""
CREATE SCHEMA IF NOT EXISTS Winter_Data_Analysis_Challenge;
COMMIT;
"""))
conn.execute(text("SET search_path TO Winter_Data_Analysis_Challenge"))

In [361]:
from sqlalchemy import inspect
inspect(db).get_schema_names()

['information_schema', 'winter_data_analysis_challenge']

In [362]:
conn.execute(text("CREATE EXTENSION IF NOT EXISTS postgis;"))

# Import data

## Life expectancy dataset

### Cleaning data

In [245]:
df_longevity =  pd.read_csv("data/life-expectancy.csv")
df_longevity.columns = ['entity', 'code', 'year', 'longevity']
df_longevity.dropna(subset = ['entity', 'year', 'longevity'], inplace = True)
df_longevity.drop_duplicates(subset = ['entity', 'year'], inplace = True)
df_longevity

,entity,code,year,longevity
0,Afghanistan,AFG,1950,27.7275
1,Afghanistan,AFG,1951,27.9634
2,Afghanistan,AFG,1952,28.4456
3,Afghanistan,AFG,1953,28.9304
4,Afghanistan,AFG,1954,29.2258
...,...,...,...,...
20750,Zimbabwe,ZWE,2017,60.7095
20751,Zimbabwe,ZWE,2018,61.4141
20752,Zimbabwe,ZWE,2019,61.2925
20753,Zimbabwe,ZWE,2020,61.1242


In [240]:
df_region = df_longevity[df_longevity['code'].isnull()]

### Longevity by continents dataframe

In [249]:
# Ignore data from "Latin America and the Carribeans" and "Northern America"
continent = ['Africa', 'Americas', 'Asia', 'Europe', 'Oceania']
df_continent = df_regions[df_regions['entity'].isin(continent)]
df_continent = df_continent.reset_index(drop = True)
df_continent.columns = ['region', 'year', 'longevity']

,region,year,longevity
0,Africa,1770,26.4000
1,Africa,1925,26.4000
2,Africa,1950,37.6235
3,Africa,1951,37.9309
4,Africa,1952,38.4430
...,...,...,...
377,Oceania,2017,78.5432
378,Oceania,2018,78.8271
379,Oceania,2019,78.7485
380,Oceania,2020,79.4958


In [294]:
conn.execute(text(f"""
DROP TABLE IF EXISTS longevity_continent CASCADE;
CREATE TABLE longevity_continent (
    region VARCHAR(15),
    year INTEGER,
    longevity FLOAT
);
"""))
df_continent.to_sql('longevity_continent', conn, if_exists='append', index=False)
query(conn, "select * from longevity_continent")


,region,year,longevity
0,Africa,1770,26.4000
1,Africa,1925,26.4000
2,Africa,1950,37.6235
3,Africa,1951,37.9309
4,Africa,1952,38.4430
...,...,...,...
377,Oceania,2017,78.5432
378,Oceania,2018,78.8271
379,Oceania,2019,78.7485
380,Oceania,2020,79.4958


### Longevity by income dataframe

In [306]:
income = ['Low-income countries', 'Lower-middle-income countries', 'Middle-income countries', 'Upper-middle-income countries', 'High-income countries', 'No income group available']
df_income = df_regions[df_regions['entity'].isin(income)]
df_income = df_income.reset_index(drop = True)
df_income.columns = ['category', 'year', 'longevity']
df_income

,category,year,longevity
0,High-income countries,1950,61.5421
1,High-income countries,1951,62.5624
2,High-income countries,1952,64.3204
3,High-income countries,1953,64.9871
4,High-income countries,1954,65.9261
...,...,...,...
427,Upper-middle-income countries,2017,75.9064
428,Upper-middle-income countries,2018,76.2589
429,Upper-middle-income countries,2019,76.4784
430,Upper-middle-income countries,2020,75.7449


In [307]:
conn.execute(text("""
DROP TABLE IF EXISTS longevity_income CASCADE;
CREATE TABLE longevity_income (
    category VARCHAR(50),
    year INTEGER,
    longevity FLOAT
);
"""))
df_income.to_sql('longevity_income', conn, if_exists='append', index=False)
query(conn, "select * from longevity_income")

,category,year,longevity
0,High-income countries,1950,61.5421
1,High-income countries,1951,62.5624
2,High-income countries,1952,64.3204
3,High-income countries,1953,64.9871
4,High-income countries,1954,65.9261
...,...,...,...
427,Upper-middle-income countries,2017,75.9064
428,Upper-middle-income countries,2018,76.2589
429,Upper-middle-income countries,2019,76.4784
430,Upper-middle-income countries,2020,75.7449


### Longevity by country development dataframe

In [324]:
development = ['Least developed countries', 'Land-locked Developing Countries (LLDC)', 'Small Island Developing States (SIDS)', 'Less developed regions, excluding least developed countries', 'More developed regions']
df_development = df_regions[df_regions['entity'].isin(development)]
df_development = df_development.reset_index(drop = True)
df_development.columns = ['category', 'year', 'longevity']

In [325]:
replacement = {'Least developed countries': 'least developed',
               'Land-locked Developing Countries (LLDC)': 'land-locked developing',
               'Small Island Developing States (SIDS)': 'small island developing',
               'Less developed regions, excluding least developed countries': 'less developed',
               'More developed regions': 'more developed'}
df_development = df_development.replace(replacement)
df_development

,category,year,longevity
0,land-locked developing,1950,39.4836
1,land-locked developing,1951,39.7985
2,land-locked developing,1952,40.2089
3,land-locked developing,1953,40.6129
4,land-locked developing,1954,40.9987
...,...,...,...
355,small island developing,2017,72.1283
356,small island developing,2018,72.2182
357,small island developing,2019,72.3776
358,small island developing,2020,72.1998


In [326]:
conn.execute(text("""
DROP TABLE IF EXISTS longevity_development CASCADE;
CREATE TABLE longevity_development (
    category VARCHAR(50),
    year INTEGER,
    longevity FLOAT
);
"""))
df_development.to_sql('longevity_development', conn, if_exists='append', index=False)
query(conn, "select * from longevity_development")

,category,year,longevity
0,land-locked developing,1950,39.4836
1,land-locked developing,1951,39.7985
2,land-locked developing,1952,40.2089
3,land-locked developing,1953,40.6129
4,land-locked developing,1954,40.9987
...,...,...,...
355,small island developing,2017,72.1283
356,small island developing,2018,72.2182
357,small island developing,2019,72.3776
358,small island developing,2020,72.1998


### Longevity by countries dataframe

In [343]:
df_country = df_longevity[~df_longevity['entity'].isin(region_value)]
df_country = df_country.drop(columns = ['entity'])
df_country = df_country.reset_index(drop = True)
df_country.columns = ['iso_code', 'year', 'longevity']

In [345]:

df_country[df_country['iso_code'].str.len() != 3]['iso_code'].unique() 

array(['OWID_KOS', 'OWID_USS', 'OWID_WRL'], dtype=object)

In [348]:
# Drop these records
df_country.drop(df_country[df_country['iso_code'].str.len() != 3].index, inplace = True)
df_country

,iso_code,year,longevity
0,AFG,1950,27.7275
1,AFG,1951,27.9634
2,AFG,1952,28.4456
3,AFG,1953,28.9304
4,AFG,1954,29.2258
...,...,...,...
19056,ZWE,2017,60.7095
19057,ZWE,2018,61.4141
19058,ZWE,2019,61.2925
19059,ZWE,2020,61.1242


In [349]:
conn.execute(text("""
DROP TABLE IF EXISTS longevity_country CASCADE;
CREATE TABLE longevity_country (
    iso_code CHAR(3) NOT NULL,
    year INTEGER,
    longevity FLOAT
);
"""))
df_country.to_sql('longevity_country', conn, if_exists='append', index=False)
query(conn, "select * from longevity_country")

,iso_code,year,longevity
0,AFG,1950,27.7275
1,AFG,1951,27.9634
2,AFG,1952,28.4456
3,AFG,1953,28.9304
4,AFG,1954,29.2258
...,...,...,...
18903,ZWE,2017,60.7095
18904,ZWE,2018,61.4141
18905,ZWE,2019,61.2925
18906,ZWE,2020,61.1242


### World longevity dataframe

In [259]:
df_world = df_longevity[df_longevity['entity'] == 'World']
df_world = df_world.drop(columns = ['entity', 'code'])
df_world = df_world.reset_index(drop = True)
df_world.columns = ['year', 'longevity']
df_world

,year,longevity
0,1770,28.5000
1,1800,28.5000
2,1820,29.0000
3,1850,29.3000
4,1870,29.7000
...,...,...
74,2017,72.3267
75,2018,72.5759
76,2019,72.7897
77,2020,72.0361


In [351]:
conn.execute(text("""
DROP TABLE IF EXISTS longevity_world CASCADE;
CREATE TABLE longevity_world (
    year INTEGER,
    longevity FLOAT
);
"""))
df_world.to_sql('longevity_world', conn, if_exists='append', index=False)
query(conn, "select * from longevity_world")

,year,longevity
0,1770,28.5000
1,1800,28.5000
2,1820,29.0000
3,1850,29.3000
4,1870,29.7000
...,...,...
74,2017,72.3267
75,2018,72.5759
76,2019,72.7897
77,2020,72.0361


## Additional dataset

### World geospatial dataframe

Link: https://public.opendatasoft.com/explore/dataset/world-administrative-boundaries/export/

In [378]:
df_geodata = gpd.read_file('data/boundaries/world-administrative-boundaries.shp')
df_geodata = df_geodata.drop(columns=['color_code', 'status', 'region', 'iso_3166_1_', 'french_shor'])
df_geodata.columns = ['iso_code', 'country', 'continent', 'geometry']
df_geodata.loc[df_geodata['country'] == 'Jersey', 'iso_code'] = 'JEY'
df_geodata = df_geodata.dropna(subset=['iso_code', 'geometry'])
df_geodata['geometry'] = df_geodata['geometry'].apply(lambda geom: MultiPolygon([geom]) if isinstance(geom, Polygon) else geom)
df_geodata = gpd.GeoDataFrame(df_geodata, geometry='geometry', crs = crs)
df_geodata = df_geodata.drop_duplicates(subset = ['iso_code'])
df_geodata = df_geodata.reset_index(drop = True)
df_geodata


,iso_code,country,continent,geometry
0,MNP,Northern Mariana Islands,Oceania,"MULTIPOLYGON (((145.63331 14.91236, 145.62412 ..."
1,FRA,France,Europe,"MULTIPOLYGON (((9.44750 42.68305, 9.45014 42.6..."
2,SRB,Serbia,Europe,"MULTIPOLYGON (((20.26102 46.11485, 20.31403 46..."
3,URY,Uruguay,Americas,"MULTIPOLYGON (((-53.37430 -33.74067, -53.39917..."
4,GUM,Guam,Oceania,"MULTIPOLYGON (((144.70941 13.23500, 144.70245 ..."
...,...,...,...,...
232,SVK,Slovakia,Europe,"MULTIPOLYGON (((22.55805 49.07944, 22.55166 49..."
233,MLI,Mali,Africa,"MULTIPOLYGON (((-4.80611 25.00027, -4.52528 24..."
234,ARM,Armenia,Asia,"MULTIPOLYGON (((46.54038 38.87559, 46.51639 38..."
235,ALB,Albania,Europe,"MULTIPOLYGON (((20.07142 42.56091, 20.10208 42..."


In [379]:
df_country_geom = df_geodata.drop(columns = ['country'])
df_country_geom

,iso_code,continent,geometry
0,MNP,Oceania,"MULTIPOLYGON (((145.63331 14.91236, 145.62412 ..."
1,FRA,Europe,"MULTIPOLYGON (((9.44750 42.68305, 9.45014 42.6..."
2,SRB,Europe,"MULTIPOLYGON (((20.26102 46.11485, 20.31403 46..."
3,URY,Americas,"MULTIPOLYGON (((-53.37430 -33.74067, -53.39917..."
4,GUM,Oceania,"MULTIPOLYGON (((144.70941 13.23500, 144.70245 ..."
...,...,...,...
232,SVK,Europe,"MULTIPOLYGON (((22.55805 49.07944, 22.55166 49..."
233,MLI,Africa,"MULTIPOLYGON (((-4.80611 25.00027, -4.52528 24..."
234,ARM,Asia,"MULTIPOLYGON (((46.54038 38.87559, 46.51639 38..."
235,ALB,Europe,"MULTIPOLYGON (((20.07142 42.56091, 20.10208 42..."


In [382]:
conn.execute(text(f"""
DROP TABLE IF EXISTS geom_country CASCADE;
CREATE TABLE geom_country (
    iso_code CHAR(3) PRIMARY KEY,
    continent CHAR(20) NOT NULL,
    geometry GEOMETRY(MULTIPOLYGON, {srid})
    )
"""))

df_country_geom.to_sql('geom_country', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})

ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'MultiPolygon'
[SQL: INSERT INTO geom_country (iso_code, continent, geometry) VALUES (%(iso_code__0)s, %(continent__0)s, %(geometry__0)s), (%(iso_code__1)s, %(continent__1)s, %(geometry__1)s), (%(iso_code__2)s, %(continent__2)s, %(geometry__2)s), (%(iso_code__3)s, %(cont ... 13602 characters truncated ... , %(continent__235)s, %(geometry__235)s), (%(iso_code__236)s, %(continent__236)s, %(geometry__236)s)]
[parameters: {'iso_code__0': 'MNP', 'geometry__0': <MULTIPOLYGON (((145.633 14.912, 145.624 14.908, 145.611 14.914, 145.573 15....>, 'continent__0': 'Oceania', 'iso_code__1': 'FRA', 'geometry__1': <MULTIPOLYGON (((9.448 42.683, 9.45 42.64, 9.476 42.579, 9.508 42.57, 9.496 ...>, 'continent__1': 'Europe', 'iso_code__2': 'SRB', 'geometry__2': <MULTIPOLYGON (((20.261 46.115, 20.314 46.07, 20.354 46.017, 20.373 45.982, ...>, 'continent__2': 'Europe', 'iso_code__3': 'URY', 'geometry__3': <MULTIPOLYGON (((-53.374 -33.741, -53.399

### Country reference dataframe

In [380]:
df_country_ref = df_geodata.drop(columns = ['continent', 'geometry'])
df_country_ref

,iso_code,country
0,MNP,Northern Mariana Islands
1,FRA,France
2,SRB,Serbia
3,URY,Uruguay
4,GUM,Guam
...,...,...
232,SVK,Slovakia
233,MLI,Mali
234,ARM,Armenia
235,ALB,Albania


In [381]:
conn.execute(text("""
DROP TABLE IF EXISTS country_ref CASCADE;
CREATE TABLE country_ref (
    iso_code CHAR(3) PRIMARY KEY,
    country VARCHAR(50)
);
"""))
df_country_ref.to_sql('country_ref', conn, if_exists='append', index=False)
query(conn, "select * from country_ref")

,iso_code,country
0,MNP,Northern Mariana Islands
1,FRA,France
2,SRB,Serbia
3,URY,Uruguay
4,GUM,Guam
...,...,...
232,SVK,Slovakia
233,MLI,Mali
234,ARM,Armenia
235,ALB,Albania
